<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [6]:
train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./sample_submission.csv")
upper_cases = [chr(ord("A") + val) for val in range(26)]

In [3]:
title_ids = list(set(train_data["Id"]))
id_title_dict = dict()
for tid in title_ids:
    id_title_dict[tid] = pd.read_json(os.path.join("./train/", tid+".json"))
    
# id_title_dict["0007f880-0a9b-492d-9a58-76eb0b0e0bd7"]

In [7]:
def find_is_datalabel(datasets, can_dataset):
    for target in datasets:
        if can_dataset.find(target) > -1 or target.find(can_dataset) > -1:
            return 1 
    return 0

id_datalabel_dict = dict()
for id_, dataset_ in zip(train_data["Id"], train_data["dataset_label"]):
    id_datalabel_dict.setdefault(id_, []).append(dataset_)

    
def find_big_case(word):
    if word.find("CoV") > -1 or word.find("COV") > -1:
        return True
    
    for l in word:
        if l not in upper_cases:
            return False
    
    return (len(word) >= 4)


def find_candidate_sen(text):
#     print(text)
#     text = "69 The Baltimore Longitudinal Study of Aging (BLSA) recently reported that the association between AD-related pathology and IR could not be confirmed."
    text = text.replace("\n", " ")
    words = text.split(" ")
    candidate_sentences = []
    candidate_datasets = []
    sentence = []
    can_dataset = []
    flag = 0
    record = False
    patience = 1
    flag_st = flag_ed = None
    for word_id, word in enumerate(words):
        if find_big_case(word):
#             print(word)
            record = True
            can_dataset.append(word)
#             can_dataset.append(" ".join(words[max(0, word_id-5):word_id+5]))
#             print(" ".join(words[max(0, word_id-5):word_id+5]))
        sentence.append(word)
      
        if len(word) > 0 and word[0] in upper_cases:
            patience = 1
            flag += 1
            if flag == 1:
                flag_st = word_id
#                 print(word)
            if flag == 2:
                record = True
                  
        else:
            if patience == 0:
                if flag > 1:
                    flag_ed = word_id
#                     print(" ".join(words[flag_st:flag_ed]))
                    can_dataset.append(" ".join(words[flag_st:flag_ed]))
                flag = 0
                patience = 1
            else:
                patience -= 1
#         print(word)

        if word.endswith(",") or word.endswith(":") or word.endswith(";"):
            if flag > 1:
                flag_ed = word_id
                can_dataset.append(" ".join(words[flag_st:flag_ed+1]))
            flag = 0
            patience = 1
        if len(word) > 0 and word[-1] in [".", "?", "!"] and (len(word) > 1 and (word[-2] not in upper_cases or len(word)>2)):
            
            if record:
                candidate_sentences.append(" ".join(sentence))
                candidate_datasets.append(can_dataset)
            can_dataset = []
            sentence = []
            record = False
            flag = 0
            patience = 1
        
#     s=2
    return candidate_sentences, candidate_datasets

In [8]:
id_dfs_dict = dict()
for tid in tqdm(title_ids):
    dfs = []
    total_row_num = id_title_dict[tid].shape[0]
    for row_id in range(total_row_num):
        candidate_sentences, candidate_datasets = find_candidate_sen(id_title_dict[tid]["text"][row_id])
#         print(candidate_sentences, candidate_datasets)
        if len(candidate_sentences)>0:
            pd_sentences, pd_datasets, labels = [], [], []
            for idx in range(len(candidate_sentences)):
                for dataset_ in candidate_datasets[idx]:
                    pd_sentences.append(candidate_sentences[idx])
                    pd_datasets.append(dataset_)
#                     print(candidate_sentences[idx], dataset_)
            
                    labels.append(find_is_datalabel(id_datalabel_dict[tid], dataset_))
            df = pd.DataFrame()
            df["Label"] = labels
            df["Sentence"] = pd_sentences
            df["Can_data"] = pd_datasets
            df["Section"] = id_title_dict[tid]["section_title"][row_id]
            df["Id"] = tid
            dfs.append(df)
    
    if dfs:
        dfs = pd.concat(dfs, axis=0)  
        id_dfs_dict[tid] = dfs
    else:
        print(tid)

can_dataset_extract = pd.concat(list(id_dfs_dict.values()), axis=0)
can_dataset_extract.to_csv("data1.csv")

 65%|██████▌   | 9335/14316 [06:46<02:16, 36.55it/s] 

55f1cc31-58e6-4ca0-b8bf-bc80bfecec09


 94%|█████████▍| 13433/14316 [09:41<00:54, 16.15it/s]

30dbdbc7-ef60-4631-86a0-131f89f353be


100%|██████████| 14316/14316 [10:21<00:00, 23.05it/s]


In [21]:
full_data = can_dataset_extract.sample(frac=1)
train_num = int(full_data.shape[0] * 0.8)
train_data = full_data.iloc[:train_num, :]
valid_data = full_data.iloc[train_num:, :]

train_data.to_csv("data1_train.csv")
valid_data.to_csv("data1_val.csv")

print(train_data.shape, (train_data["Label"]==1).mean())
print(valid_data.shape, (valid_data["Label"]==1).mean())

In [22]:
train_data

,Label,Sentence,Can_data,Section,Id
4,0.0,An R-squared value of 0.77 indicates that 77% ...,An R-squared value,Descriptive Statistics,56c6c278-127f-4f22-8987-397807be5288
1,0.0,Examples are the Alzheimer's Disease Neuroimag...,Autism Centers of Excellence (ACE),Introduction,dabd87cc-78e4-4f6f-80e9-8cf0536473a0
20,0.0,Additional considerations were examined during...,ISSN,Journal of Educational Issues,4805591b-7868-4998-8459-ed5ee29934bd
31393,0.0,"Disclosure: J. Braun, None; H. Kellner, None;...","J. Braun,",,c12a3ad2-17bc-4b8b-8779-77ba2680ae96
5,0.0,"2), we now mainly observe a direct transition ...",Hubei to Italy (posterior,Unsampled diversity reinforces reconstructions...,2670cc05-996e-4db3-99b4-baa5eef226e8
...,...,...,...,...,...
4,0.0,Coupled with long-term growth of milk producti...,"Washington State,",,cd0ff252-2e06-4be5-ad54-a321df38ec2f
4,0.0,We create four classes (Table 2) based on CDR ...,"CDR-0 (No Dementia),",Data Selection,0768674c-23dd-4aeb-980b-d310005a63f6
3,0.0,Bu çalışmalara örnek olarak PISA (Programme fo...,TIMSS,Türkçe Sürüm Giriş,95d21d82-310e-4ce7-9e5a-30bbdec1eb8d
0,0.0,As the three clustering coefficients refer to ...,Supplementary Text S2 for,Feature Extraction Based on Clustering Coeffic...,f5ba4dec-9b18-4410-af7a-af7bcd894e70


In [159]:
# check 
# print(can_dataset_extract["Label"].mean())
# can_dataset_extract[can_dataset_extract["Label"]==1]

# list(set(train_data["dataset_label"]))

In [161]:
# check if omit some data label

# non_found_dataset = []
# for dataset in list(set(train_data["dataset_label"])):
# #     print("======")
# #     print(dataset)
#     flag = 0
#     for val in can_dataset_extract["Can_data"].values:
#         if val.find(dataset) > -1 or dataset.find(val) > -1:
# #             print(val)
#             flag = 1
#             break
#     if flag == 0:
#         non_found_dataset.append(dataset)

# len(non_found_dataset), non_found_dataset

In [166]:
# counted = 0
# for idex, tid in enumerate(title_ids):
#     for val in id_title_dict[tid]["section_title"].values:
#         if str(val).find("Data") > -1 or str(val).find("data") > -1 :
#             counted += 1
#             print(idex)
#             break
# counted